Educational materials on QUBO formulation.

# 1. What problems do quantum annealers solve?

Quantum annealing algorithm solves a quadratic unconstrained binary optimization problem, or equivalently, an Ising problem (or Ising model).

### 1.1. Quadratic unconstrained binary optimization problem

A quadratic unconstrained binary optimization (QUBO) problem is a combinatorial optimization problem with the following characteristics:

1. The variables are binary $\in\left\{0,1\right\}$.

2. The objective function is a quadratic polynomial. It is a sum of __ONLY__ second-order and first-order terms of the variables, and a constant term.

3. There is no constraint.

#### 1.1.1. Objective function of a QUBO problem

A QUBO problem, with $N$ binary variables, can be described by a minimization (or maximization, by multiplying the objective function by $-1$) of a quadratic objective function of the variables as the following:

\begin{equation}
\min_{x_i\in\{0,1\}\\i=1,...,N}\sum\limits_{i,j=1}^N Q_{i,j} x_i x_j +C,
\end{equation}

where $Q_{i,j}\in\mathbb{R},\forall i,j=1,...,N$, and $C\in\mathbb{R}$ are all constants.

A general form of the objective function, of $N$ binary variables, is

\begin{equation}
f({\bf x})=\sum\limits_{i,j=1}^N Q_{i,j} x_i x_j +C,~\text{ where }{\bf x}=(x_1,...,x_N)\in\{0,1\}^N.
\end{equation}

Equivalently,

\begin{equation}
f({\bf x})=\mathbf{x}Q\mathbf{x}^\intercal +C,~\text{ where }{\bf x}=(x_1,...,x_N)\in\{0,1\}^N,
\end{equation}

where $Q$ is an $N\times N$ matrix, in which $Q_{i,j}$ is the $(i,j)$-th element of $Q$, $\forall i,j=1,...,N$.

$Q$ is often called the "QUBO matrix" of the problem. In fact,

\begin{equation}
\underset{x_i\in\{0,1\}\\i=1,...,N}{\operatorname{argmin}}\mathbf{x}Q\mathbf{x}^\intercal+C
=\underset{x_i\in\{0,1\}\\i=1,...,N}{\operatorname{argmin}}\mathbf{x}Q\mathbf{x}^\intercal,
\end{equation}

and hence the QUBO matrix fully describes the goal of the optimization problem on its own - which is to find the best variable settings for optimizing the objective function.

#### 1.1.2. Question: where are the first-order terms?

Since the variables $x_i$'s are $\in\{0,1\}$,

\begin{equation}
x_j^2 = x_j.
\end{equation}

Due to this property, the objective function can be written as:

\begin{equation}
f({\bf x})=\sum\limits_{i,j=1\\i\neq j}^N Q_{i,j} x_i x_j + \sum\limits_{i=1}^N Q_{i,i} x_i + C,
\end{equation}

and it is observed that the first-order terms are in the second term of the above. Hence, the diagonal elements in $Q$ are the coefficients of the first-order terms!

#### 1.1.3. A more compact form of the objective function

Observe that

\begin{align}
f({\bf x})&=\sum\limits_{i,j=1\\i\neq j}^N Q_{i,j} x_i x_j + \sum\limits_{i=1}^N Q_{i,i} x_i + C,\\
&=\sum\limits_{i,j=1\\i<j}^N \left(Q_{i,j}+Q_{j,i}\right) x_i x_j + \sum\limits_{i=1}^N Q_{i,i} x_i + C.
\end{align}

If we define $M_{i,j}$'s to be:

\begin{equation}
M_{i,j}=\begin{cases}
Q_{i,i} &,\text{if } i=j\\
Q_{i,j}+Q_{j,i} &,\text{if } i<j\\
0 &,\text{if } i>j
\end{cases}
\end{equation}

Then, 
\begin{equation}
f({\bf x})=\sum\limits_{i,j=1\\i<j}^N M_{i,j} x_i x_j + \sum\limits_{i=1}^N M_{i,i} x_i + C,
\end{equation}

and the objective function can be described by a (upper-) triangular matrix $M$!

This is important from a programming standpoint, as eliminating all lower-triangular terms of  the QUBO matrix reduces the number of terms in the objective function, and hence reduces the complexity.

### 1.2. Ising problem

In Ising problem, all charateristics are the same to QUBO problem, __EXCEPT__ that the The variables are in $\left\{-1,1\right\}$, instead of $\{0,1\}$.

A mapping between a QUBO formulation and Ising formulation of a problem can be regarded as finding a 1-1 mapping between the two model, and this can be done by either of the following:

\begin{equation}
(0,1)\longleftrightarrow (1,-1),\text{ which can be mapped by }x\longleftrightarrow z=-2x+1,
\end{equation}

or

\begin{equation}
(0,1)\longleftrightarrow (-1,1),\text{ which can be mapped by }x\longleftrightarrow z=2x-1.
\end{equation}

Due to the existence of 1-1 linear mapping between the variables in the two problems, they are equivalent (or more rigorously, the two expressions are isomorphic).

### 1.3. D-Wave specific

In D-Wave Ocean SDK, the QUBO problem is encoded in a dictionary with items

\begin{equation}
(i,j):Q_{i,j},
\end{equation}

in which $i,j\in0\cup\mathbb{N}$, representating a cross term of the $i$-th and $j$-th binary variables, that is,

\begin{equation}
Q_{i,j}x_i x_j.
\end{equation}


# 2. Reduce a combinatorial optimization problem to a QUBO problem

To be able to solve a combinatorial optimization problem using quantum annealing, one needs to find a reduction mapping from the problem to a QUBO problem. The reduction mapping ensures that the QUBO problem being solved implies the original problem is solved as well.

A typical class of optimization problems in which most combinatorial optimization problems can  be formulated into has its problem in the following form:

\begin{equation}
\min_{x\in\mathbf{D}} f({\bf x}),~\text{in which $D$ is a discrete space},
\end{equation}

w.r.t. $n_\text{eq}$ equality and $n_\text{ineq}$ constraints:

\begin{equation}
g_i({\bf x})= a_i,~\text{for }i=1,...,n_\text{eq},
\end{equation}

\begin{equation}
0\leq f_j({\bf x})\leq b_j,~\text{for }j=1,...,n_\text{ineq}.
\end{equation}

The following items need to be addressed in order to put the problem in a QUBO form:

1. The variables in discrete space $D$ need to be mapped to binary variables in some product space of $\{0,1\}$.
2. The constraints need to be removed.
3. The objective function in the end need to be a quadratic polynomial of the binary variables.

We will talk about the remedy to these sequentially in the following sections; yet, before we move on, let's talk about a trick that is useful in addressing inequality constraints.

###  2.1. Addressing inequality constraints with at most a logarithmic amount of overhead (additional binary variables)

Without loss of generality, assume we have a function $f(\bf{x})$ of a vector of binary variables $\bf{x}$, and the constraint is:

\begin{equation}\label{eq:1.00}
0\leq f({\bf x})\leq M,
\end{equation}

where $M\in\mathbb{N}$.

__Case 1__

If $M+1=2^n$ for some $n\in\mathbb{N}$, and $f({\bf x})$ has range equal to $\{0,1,...,M\}$, then $0\leq f({\bf x})\leq M$ is automatically implied, and there is nothing that needs to be done.

An example of this case would be that $f({\bf x})=\sum\limits_{i=1}^{log_2(M+1)}x_i 2^{i-1}$.

__Case 2__ 

If $M+1\neq2^n$ for all $n\in\mathbb{N}$, the following method can be used.

Let $a_i\in\{0,1\}$, for $i=1,...,\lceil\log_2(M+1)\rceil$, be binary variables, and ${\bf a}\equiv\left(a_1,...,a_{\lceil\log_2(M+1)\rceil}\right)$, let

\begin{equation}
g({\bf a})=\sum\limits_{i=1}^{\lceil\log_2(M+1)\rceil}a_i 2^{i-1},
\end{equation}

such that $g(\bf{a})$ is an integer in which its binary representation is ${\bf a}$. Then, by introducing the equality constraint

\begin{equation}
f({\bf x})+g({\bf a}) = M,
\end{equation}

this implies $0\leq f({\bf x})\leq M$, since

\begin{equation}
0\leq f({\bf x}) = M-g({\bf a})\leq M.
\end{equation}

### 2.2. Item 1: The variables in discrete space $D$ need to be mapped to binary variables in some product space of $\{0,1\}$

Often times, $D$ is a product space of integer intervals, i.e. $D=I_1\times I_2\times...\times I_N$, where for each $i\in\{1,2,...,N\}$, $I_i=\{L_i,L_i+1,...,U_i\}$, for some $L_i,U_i\in\mathbb{N}$ such that $L_i<U_i$.

One can always shift the variables by a constant such that $L_i=0$ for all $i$, and therefore, without loss of generality, we will assume this. Then, for each variable $x_i$,

\begin{equation}
x_i\in\{0,1,...,U_i\}.
\end{equation}

Now, we replace $x_i$ with a $\lceil\log_2(U_i+1)\rceil$ binary variables that can be used to express it. Introduce the new binary variables

\begin{equation}
x_{i,j}\in\{0,1\},~\text{for }j=1,...,\lceil\log_2(U_i+1)\rceil,
\end{equation}

then

\begin{equation}
\sum\limits_{j=1}^{\lceil\log_2(U_i+1)\rceil}x_{i,j} 2^{j-1}
\end{equation}

is a binary representation that covers the interval $\left[0,U_i\right]$. Then, we can replace $x_i$ with $x_{i,j}$'s by establishing

\begin{equation}
x_i=\sum\limits_{j=1}^{\lceil\log_2(U_i+1)\rceil}x_{i,j} 2^{j-1},
\end{equation}

with the constraint

\begin{equation}
0\leq\sum\limits_{j=1}^{\lceil\log_2(U_i+1)\rceil}x_{i,j} 2^{j-1}\leq U_i.
\end{equation}

This inequality constraint can be taken care of following the method in Sec.2.1.

### 2.3. Item 2: The constraints need to be removed

#### 2.3.1. Removal of equality constraints

For each equality constraint $g_i({\bf x})= a_i,~\text{for }i=1,...,n_\text{eq}$, we add a penalty

\begin{equation}
P_i(g_i({\bf x})-a_i)^2,
\end{equation}

in which $P_i\ll 0$ is a large penalty constant, to the objective function. The effect of this is that if the constraint is violated,

\begin{equation}
(g_i({\bf x})-a_i)^2 > 0,
\end{equation}

and there will be a huge penalty $P_i(g_i({\bf x})-a_i)^2$ showing up in the objective value, in which the {\bf x} will have a high objective value, such that it is far from an optimal solution (in which the objective value is minimized).

Typically, we pick $P_i=P$ for all $i$ for some fixed $P$; yet, we wish to emphasize that the penalty values for different equality constraints can be different.

#### 2.3.2. Removal of inequality constraints

For each inequality constraint $0\leq f_j({\bf x})\leq b_j,~\text{for }j=1,...,n_\text{ineq}$, follow these two steps:

1) Map the inequality constraint to equality constraint by the method in Sec.2.1.

2) Remove the corresponding equality constraint by the method in Sec.2.3.1.

### 2.4. Item 3: The objective function in the end need to be a quadratic polynomial of the binary variables

After all prior steps were taken, we should have an unconstrainted binary optimization problem with an objective function alone.

__Case 1 : If the objective function is a polynomial of the variables__

In this case, if the order of the polynomial is $\leq 2$, then we don't need to do anything, and the problem is a QUBO problem.

If the order of the polynomial is $>2$, then we can reduce the order to $\leq 2$ by using Rosenberg's polynomials. We won't specify the method here, but references include [Ref.1](https://arxiv.org/pdf/2001.00658.pdf),[Ref.2](https://arxiv.org/abs/1806.08815). Note that this will result in additional binary variables, which increase the complexity of the problem.

__Case 2 : If the objective function is NOT a polynomial of the variables__

In this case, the best one can do is to approximate the objective function with some, potentially high-order, polynomials. Afterwards, the QUBO problem can  be generated following the method in __Case 1__.

# 3. Examples

Let's look at some examples!

1) Kidney exchange

2) Player management

3) Cost optimization

# 4.  Let's work out our problem, and solve with D-Wave's LeapHybrid solver! (work in progress)

### 4.1. Problem formulation

Let there be $N\in\mathbb{N}$ locations (e.g. medical centers in a pandemic) in a resource distribution network.

In this scenario that we consider, the goal is to determine an efficient strategy for equally grouping the locations into $k\in\mathbb{N}$ groups, where $0\equiv N\text{ mod }k$, such that the resources could be best exchanged.

The intent is to generate an undirected graph with weighted nodes, and formulate the problem as finding the maximum-weighted $k-$clique on the graph. We attempt to explain more in detail about the model in Sec.4.2.

### 4.2. A general model

Continuing from the previous descriptions, let $\{l_i\}_{i=1}^N$ be the set of $N$ locations, and we wish to separate them into groups of $\frac{N}{k}$. There are ${N\choose k}$ such possible groupings, and we let each unique grouping be described by an $\frac{N}{k}$-tuple from the set:

\begin{equation}
V=\left\{(l_{j_1},...,l_{j_\frac{N}{k}}):j_i\in\left\{1,...,N\right\}\text{ and }j_i<j_k,\forall i,k\in\left\{1,...,\frac{N}{k}\right\}\text{ and } i<k\right\}.
\end{equation}

Furthermore, for each grouping $v\in V$, a utility value $U(v)\in\mathbb{R}$ is assigned to it. We don't specify the detail about the utility function $U(\cdot)$, and we leave it to be general as it is. The design of a proper $U(\cdot)$ would be different case by case, and it is dependent on the locations that are involved.

Now some of the groups may have low utilities that we don't wish such groupings to be considered. Therefore, it is useful to perform a pre-filtering to screen out those groups. Having $v\in V$ and the respective $U(v)$, say, we can set a threshold $U_{th}\in\mathbb{R}$ such that only groups with $U(v)\geq U_{th}$ are allowed. We denote the set of allowed groups to be $\tilde{V}\subset V$. In this case,

\begin{equation}
\tilde{V} = \left\{v\in V:U(v)\geq U_{th}\right\}.
\end{equation}

As we will see, this kind of pre-filtering is helpful in reducing the size of the respective graph and hence reducing the complexity of the problem. 

Now we are ready to define the targeted graph. The set of vertices is simply $\tilde{V}$. The set of undirected edges, $E$, is defined as some pairs of the end-vertices:

\begin{equation}
E = \left\{(v_1,v_2): v_1,v_2\in{\tilde{V}}\text{ and }{v_1}_i\neq {v_2}_j,\forall i,j\in\left\{1,...,\frac{N}{k}\right\} \right\},
\end{equation}

in which an edge exists between a pair of vertices iff. they do not share any same location.

Having the graph $G=(\tilde{V},E)$, we wish to find the $k$-clique with the maximum sum of the utilities of its vertices. This can be modeled as a maximum-weighted $k$-clique problem on $G$, and we could perform a reduction map similar to~\cite{Bass18} to map the problem to a QUBO problem, and solve it with relevant quantum computational methods.

### 4.3. A model for utilities and pre-filtering that maximizes the number of possible resource transfer

In Sec.4.2., we proposed the general form of our model, without specifying the mechanisms for utilities and pre-filtering. Here, we propose a simple model for them.

Consider only one type of resource, with the quantity of the resource exists in increments of $1$, i.e. integer values. For each location $l_i$, let $S_i\in\mathbb{Z}$ be the number of available resource at that location. $S_i>0$ means a surplus of the resource at location $l_i$, in which the location can offer to other locations with shortage; $S_i<0$ means a shortage of the resource at location $l_i$, and $\lvert S_i\rvert$ in this case is the demand.

Now, for each grouping $v\in V$, let
\begin{equation}
v=(l_{j_1},...,l_{j_\frac{N}{k}})\text{, where }j_i\in\left\{1,...,N\right\}\text{ and }j_i<j_k,\forall i,k\in\left\{1,...,\frac{N}{k}\right\}\text{ and } i<k.
\end{equation}
Note that the elements in order in $v$ can be regarded as a length-$\frac{N}{k}$ subsequence of the elements in order in $(l_1,...,l_N)$.

Now, we separate the locations in $v$ into three subsets by the values $S_{j_i}$'s. Let
\begin{equation}
L_v^{(+)} = \left\{j:S_j\geq0\text{ and }l_j\text{ is a component of } v\right\},
\end{equation}
\begin{equation}
L_v^{(-)} = \left\{j:S_j<0\text{ and }l_j\text{ is a component of } v\right\},
\end{equation}
\begin{equation}
L_v^{(0)} = \left\{j:S_j=0\text{ and }l_j\text{ is a component of } v\right\}.
\end{equation}

Then, we define the utility of $v$ to be
\begin{equation}
U_1(v)\equiv \min\left\{\sum\limits_{i\in L_v^{(+)}}S_{i}~,\sum\limits_{i\in L_v^{(-)}}\lvert S_{i}\rvert\right\},
\end{equation}
and this is the total number of resources that could be transferred from locations  with surplus to locations with shortage within the locations in $v$. Note that in the case the set $L_v^{(+)}$, or $L_v^{(-)}$, is empty, the corresponding sum is $0$. In the end, a $K$-clique with a maximum sum of its vertices' utility weights correspond to a grouping strategy such that the transfer of resources is maximized. This is the goal of this particular model.

The pre-filtering process would be to pick a reasonable $U_{th}\in\mathbb{R}$ as stated in Sec.4.2. Figuring out a good value of $U_{th}$ is important; $U_{th}$ being too low would result in no reduction in the number of  vertices, and therefore no reduction in the complexity of the graph; $U_{th}$ being too high would result in too many vertices being filtered out, and as a result, no or too few $k$-cliques on the graph as options in latter optimization.

We would also like to point out that in the case where the majority of locations is suffering from shortage, then a good grouping strategy may have some of its vertices having $0$ utility! Then, setting $U_{th}=0$ is needed, and this would correspond to no pre-filtering.

### 4.4. Maximum $k$-clique problem on $G=(\tilde{V},E)$

Let the vertices in $\tilde{V}$ be labeled by index $i\in\{1,2,...,\lvert\tilde{V}\rvert\}$, and let the binary variables

\begin{equation}
x_i=\begin{cases}
1&,~\text{if grouping }v_i~\text{is selected}, \\ 
0&,~\text{otherwise}.
\end{cases}
\end{equation}

The objective is then

\begin{equation}
f({\bf x})=-\sum_i U(v_i)x_i,
\end{equation}

w.r.t. constraints

\begin{equation}
\sum_i x_i=k,
\end{equation}

and the edge set $E$.

Here's an example of a resulting QUBO objective:

\begin{equation}
f({\bf x})=-\sum_i U(v_i)x_i+P_1(\sum_i x_i-k)^2+P_2\sum\limits_{i<j\\(v_i,v_j)\notin E}x_i x_j,
\end{equation}